## IO

In [1]:
# -----------------------
# Import statements
# -----------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms 
from torch.utils.data import DataLoader
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [2]:
# GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)

GPU State: cpu


In [3]:
def training_loop(model, loss, optimizer, loader, epochs, verbose=True, device=device): 
    """
    Run training of a model given a loss function, optimizer and a set of training and validation data.
    """
    
    # Train 
    for epoch in range(epochs): 
        for times, data in enumerate(loader): 
            inputs, labels = data[0].to(device), data[1].to(device)

            # Zero the parameter gradients 
            optimizer.zero_grad()
            
            # Forward and backward + optimize 
            outputs = model(inputs)
            loss_tensor = loss(outputs, labels)
            loss_tensor.backward()
            optimizer.step()
            
            # Print statistics 
            if verbose and (times % 100 == 99):
                print('[%d/%d, %d/%d] loss: %.3f' %
                    (epoch+1, epochs, times+1, len(loader),
                    loss_tensor.item()))
         

In [15]:
def evaluate_model(model, loader, device=device): 
    """
    Evaluate a model 'model' on all batches of a torch DataLoader 'data_loader'.

    Returns: the total number of correct classifications,
             the total number of images
             the list of the per class correct classification,
             the list of the per class total number of images.
    """
    
    # test
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            # For each sample pick the class with largest score across the class dimension (dim=1)
            # Predicted has the shape of [batch_size] and contains class indices (e.g 0-9)
            _, predicted = torch.max(outputs.data, dim = 1)
            total += labels.size(0)     # tensor(57, device='cuda:0') 
            correct += (predicted == labels).sum().item()   # The .item make 57 to an int
            
    class_correct = [0 for i in range(10)]
    class_total = [0 for i in range(10)]
        
    with torch.no_grad():
        for data in loader: 
            inputs, labels = data[0].to(device), data[1].to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, dim = 1)
            # squeeze removes the dim = 1 so it becomes a boolean 
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
    return (correct, total, class_correct, class_total)
    

In [11]:
def get_train_transforms():
    """Return a normalized version of the trainingset"""
    return transforms.Compose(
        [
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=(0.4914, 0.4822, 0.4465),
                std=(0.2470, 0.2435, 0.2616),
            ),
        ]
    )

def get_test_transforms():
    """Return a normalized version of testset"""
    return transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(
                mean=(0.4914, 0.4822, 0.4465),
                std=(0.2470, 0.2435, 0.2616),
            ),
        ]
    )

train_ds = datasets.CIFAR10(
        root="data", train=True, download=True, transform=get_train_transforms())
test_ds = datasets.CIFAR10(
        root="data", train=False, download=True, transform=get_test_transforms())
train_loader = DataLoader(dataset=train_ds, batch_size=64, shuffle=True, num_workers=0,)
test_loader = DataLoader(dataset=test_ds, batch_size=64, num_workers=0)


Files already downloaded and verified
Files already downloaded and verified


### Model

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, (3,3), (1,1), (1, 1))
        self.conv2 = nn.Conv2d(16, 32, (3,3), (1,1), (1, 1))
        self.fc1 = nn.Linear(32*8*8, 256)
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x): 
        x = self.conv1(x)                                         # (N, 16, 32, 32)
        x = F.relu(x)                                       # (N, 16, 32, 32)
        x = F.max_pool2d(x, kernel_size=2, stride=2)        # (N, 16, 16, 16)
        x = self.conv2(x)                                         # (N, 32, 16, 16)
        x = F.relu(x)                                       # (N, 32, 16, 16)
        x = F.max_pool2d(x, kernel_size= 2, stride = 2)     # (N, 32, 8, 8)
        x = torch.flatten(x, start_dim=1)                   # (N, 2048)
        x = self.fc1(x)                                           # (N, 256)
        x = F.relu(x)                                       # (N, 256)
        x = self.fc2(x)                                           # (N, 10)
        return x                                                  # logits

# Make the model 
net = Net().to(device)
print(net)

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


In [17]:
# Parameters 
epochs = 4
lr = 2e-3
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.002, momentum=0.9)

# Train
print('Training on %d images' % train_ds.data.shape[0])
training_loop(net, loss, optimizer, train_loader, epochs)
print('Training Finished.\n')

# Test
correct, total, class_correct, class_total = evaluate_model(net, test_loader)
print('Accuracy of the network on the %d test images: %d %%' % (test_ds.data.shape[0], (100*correct / total)))
for i in range(10):
    print('Accuracy of %d: %3f' % (i, (class_correct[i]/class_total[i])))

Training on 50000 images
[1/4, 100/782] loss: 1.047
[1/4, 200/782] loss: 1.090
[1/4, 300/782] loss: 0.842
[1/4, 400/782] loss: 1.124
[1/4, 500/782] loss: 1.233
[1/4, 600/782] loss: 1.143
[1/4, 700/782] loss: 1.120
[2/4, 100/782] loss: 0.871
[2/4, 200/782] loss: 1.001
[2/4, 300/782] loss: 0.906
[2/4, 400/782] loss: 1.111
[2/4, 500/782] loss: 1.026
[2/4, 600/782] loss: 1.157
[2/4, 700/782] loss: 0.958
[3/4, 100/782] loss: 0.986
[3/4, 200/782] loss: 1.086
[3/4, 300/782] loss: 0.754
[3/4, 400/782] loss: 0.969
[3/4, 500/782] loss: 0.760
[3/4, 600/782] loss: 1.198
[3/4, 700/782] loss: 0.820
[4/4, 100/782] loss: 0.878
[4/4, 200/782] loss: 0.924
[4/4, 300/782] loss: 1.059
[4/4, 400/782] loss: 1.018
[4/4, 500/782] loss: 0.969
[4/4, 600/782] loss: 0.852
[4/4, 700/782] loss: 0.986
Training Finished.

Accuracy of the network on the 10000 test images: 69 %
Accuracy of 0: 0.796000
Accuracy of 1: 0.830000
Accuracy of 2: 0.611000
Accuracy of 3: 0.429000
Accuracy of 4: 0.537000
Accuracy of 5: 0.725000
